<a href="https://colab.research.google.com/github/xander-bennett/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/Rob1Ham/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/Rob1Ham/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be sometime that can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [0]:
import pandas as pd
import numpy as np
import glob

#setting the filepaths for all the images for forests and mountains
mountains = glob.glob("data/mountain/*.jpg")
forests = glob.glob("data/forest/*.jpg")

In [0]:
def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

## Instatiate Model

In [3]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model # This is the functional API
from skimage import io, transform

resnet = ResNet50(input_shape=(224, 224, 3),weights='imagenet', include_top=False)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 42s 0us/step


In [0]:
#preventing the model that is being imported from being retrainable
for layer in resnet.layers:
    layer.trainable = False

In [5]:
#taking the output layer and setting it to variable X
#then applying the GlobalAveragePooling flattening function to it
#then applying a dense layer on top of that
#then creating a predictions layer with 1 node as our output
#finally instantiating a model using our previous imported model as an input, and the prediction variable as the output
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=resnet.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
def process_img_to_array(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [7]:
import os

#creating a list of filepaths for all the images of mountains and forests

forest = [x for x in os.listdir('./data/forest') if x[-3:] == 'jpg']

mountains = [x for x in os.listdir('./data/mountain') if x[-3:] == 'jpg']


#assigning forrests to 0, and mountains to 1.
zeros = np.zeros(len(forest))
ones = np.ones(len(mountains))

#appending the two variables to a y variable
y = np.append(zeros, ones)
y.shape

FileNotFoundError: ignored

In [0]:
#reshaping the array so there is one column
y = y.reshape(-1,1)

In [0]:
y.shape

In [0]:
#creating X variables of numpy arrays for each image in the filepath.

data = []
for i in ['forest', 'mountain']:
    for file in os.listdir('./data/'+i):
        #only images
        if file[-3:] == 'jpg':
            #filepath for images
            path = os.path.join(f'./data/{i}/' + file)
            #transforming the image
            img = process_img_path(path)
            #converting to array
            x = image.img_to_array(img)
            #expanding dimensions, preprocessing, and appending to the data variable
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            data.append(x)

In [0]:
#reshaping the data array to input into the developed model
X =np.asarray(data).reshape(len(data),224,224,3)

## Fit Model

# Custom CNN Model

## Fit Model

In [0]:
model.fit(X, y, epochs=10, batch_size=10, validation_split=0.1)

In [0]:
#lets test an image:

In [0]:
!pwd

<img align="left" src="https://c402277.ssl.cf1.rackcdn.com/photos/2325/images/hero_small/mountains-hero.jpg" width=200>

In [0]:
!wget https://c402277.ssl.cf1.rackcdn.com/photos/2325/images/hero_small/mountains-hero.jpg

In [0]:
!wget https://ca.slack-edge.com/T4JUEB3ME-ULJ9DTDKL-246bfe8730a9-512

In [0]:
test = 'T4JUEB3ME-ULJ9DTDKL-246bfe8730a9-512.jpg'

In [0]:
test = 'mountains-hero.jpg'

In [0]:
test_data = []
if file[-3:] == 'jpg':
    path = "./"+test
    img = process_img_path(path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_data.append(x)

In [0]:
model.predict(test_data)

In [0]:
model.evaluate(X, y)

# Custom CNN Model

In [0]:
# Compile Model


In [0]:
# Compile Model


In [0]:
# Fit Model


# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language